In [2]:
import numpy as np
import scipy.fftpack as fftpack
import matplotlib.pyplot as plt
import cic
%load_ext Cython

In [2]:
pos = cic.read_position_data(cic.SNAPPREFIX)

In [3]:
vel = cic.read_velocity_data(cic.SNAPPREFIX)

In [4]:
points, _, ndims = cic.calculate_params(pos)

In [4]:
%%cython -a
from __future__ import division, print_function
import math
import numpy as np
cimport numpy as cnp
cimport cython

# Define types to be used as array dtypes
FLOAT = np.float
INT = np.int_
UINT = np.uint
ctypedef cnp.float_t FLOAT_t
ctypedef cnp.int_t INT_t
ctypedef cnp.uint_t UINT_t

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef cnp.ndarray[FLOAT_t, ndim=4] cic_3d(FLOAT_t[:, :] points, INT_t[:] ndims, FLOAT_t[:, :] weights=None):
    """ A Cython-based 3D cloud-in-cell algorithm.

    Parameters
    ----------
    points : cnp.float_t[:, :]
        Must be non-negative. `points.shape[1]` must be 3.
    ndims : cnp.int_t[:]
        Desired number of cells per dimension. Must have length 3.
    weights : optional
        Weight for each point. There must be a weight for each point.

    Returns
    -------
    ndensity : np.ndarray[FLOAT_t, ndim=2]
        A 2D array of the number density per cell. It has shape (ndims + 2).

    Raises
    ------
    ValueError
        If any of the following conditions are not met:
        points.shape[1] == ndims.shape[0]
        points.shape[1] == 3
        points.shape[1] == ndims.shape[0]
        (ndims > 0).all()
        (points <= ndims).all() and (points > 0).all()
    """
    cic_sanitize(points, ndims)
    if ndims.shape[0] != 3:
        raise ValueError("Argument `ndims` must be of shape (3,).")
    cdef UINT_t space = 3
    # TODO: Check weights sanity: non-negative & share shape[0] with points
    if weights is None:
        weights = np.ones((points.shape[0],1), dtype=FLOAT)

    # Pre-allocate variables to be used in the main loop
    # ndensity has shape ndims + 2 in case points are within 0.5 of the box edge.
    cdef cnp.ndarray[FLOAT_t, ndim=4] ndensity = np.zeros((ndims[0]+2, ndims[1]+2, ndims[2]+2, weights.shape[1]), dtype=FLOAT)
    cdef UINT_t[:] cell = np.zeros(space, dtype=UINT)
    cdef FLOAT_t[:, :] delta = np.zeros((2, space), dtype=FLOAT)
    cdef FLOAT_t celldelta = 0
    cdef FLOAT_t cellvol = 0
    cdef UINT_t i, j, k, w
    cdef size_t n

    # Loop over the number of points
    for n in range(points.shape[0]):
        # Loop over each dimension of each point
        for i in range(space):
            cell[i] = <UINT_t>(points[n, i] + 0.5)
            celldelta = cell[i] - points[n, i]
            delta[0, i] = 0.5 + celldelta
            delta[1, i] = 0.5 - celldelta
        # Loop over the 8 neighbors of each point
        for i in range(2):
            for j in range(2):
                for k in range(2):
                    cellvol = delta[i,0] * delta[j,1] * delta[k,2]
                    # Loop over each component of the weight
                    for w in range(weights.shape[1]):
                        ndensity[cell[0]+i, cell[1]+j, cell[2]+k, w] += cellvol * weights[n, w]

    return ndensity

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef cnp.ndarray[FLOAT_t, ndim=4] wrap_periodic_3d(FLOAT_t[:, :, :, :] field):
    cdef size_t x_len = field.shape[0]
    cdef size_t y_len = field.shape[1]
    cdef size_t z_len = field.shape[2]
    cdef size_t ndim = field.shape[3]
    cdef size_t x, y, z, i
    for x in range(x_len):
        for y in range(y_len):
            for z in range(z_len):
                for i in range(ndim):
                    pass

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef int cic_sanitize(FLOAT_t[:, :] points, INT_t[:] ndims):
    """Checks inputs to cloud-in-cell functions for sanity.

    Parameters
    ----------
    points : a 2D array of floats.
    ndims : a 1D array of integers.

    Returns
    -------
    int
        True if all test passes

    Raises
    ------
    ValueError
        If any of the following conditions are not met:
        points.shape[1] == ndims.shape[0]
        (ndims > 0).all()
        (points <= ndims).all() and (points > 0).all()
    """
    if points.shape[1] != ndims.shape[0]:
        raise ValueError("Dimension mismatch between arguements `points` and `ndims`.")

    cdef size_t i, j
    for i in range(ndims.shape[0]):
        if ndims[i] <= 0:
            raise ValueError("Argument `ndims` must be positive.")
    for i in range(points.shape[0]):
        for j in range(points.shape[1]):
            if points[i,j] > ndims[j]:
                raise ValueError("Argument `ndims` too small for the range of `points`")
            elif points[i,j] < 0:
                raise ValueError("Argument `points` must be non-negative.")
    return True

In [7]:
timeit cic.cic(points[:100000],ndims)

1 loops, best of 3: 9.65 s per loop


In [7]:
timeit cic_3d(points[:1000],ndims)

100 loops, best of 3: 1.92 ms per loop


In [6]:
timeit cic_3d(points[:100000],ndims)

100 loops, best of 3: 7.03 ms per loop


In [11]:
np.abs(cic.cic(points[500:1500],ndims) - cic_3d(points[500:1500],ndims).reshape((130,130,130))).sum()

0.0

In [117]:
testvector = np.array((1.125, 1.5, 1.675, 2.25, 2.5))
testcell = np.array((1, 2, 2, 2, 2))
testdelta = 0.5 + testcell - testvector
testdelta

array([ 0.375,  1.   ,  0.825,  0.25 ,  0.   ])

In [103]:
def correct_delta(p):
    return 0.5 + np.rint(p) - p

In [104]:
correct_delta(testvector) == testdelta

array([ True,  True,  True,  True,  True], dtype=bool)

In [119]:
1 - (testvector + 0.5) % 1

array([ 0.375,  1.   ,  0.825,  0.25 ,  1.   ])

In [159]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

INT = np.uint
ctypedef np.uint_t INT_t
@cython.wraparound(False)
@cython.boundscheck(False)
def cython_delta(double[:] p):
    cdef np.ndarray result = np.zeros(p.shape[0], dtype=INT)
    cdef INT_t[:] result_view = result
    cdef size_t i
    for i in range(p.shape[0]):
        result_view[i] = <INT_t>(p[i] + 0.5)
    return result

In [160]:
cython_delta(testvector)

array([1, 2, 2, 2, 3], dtype=uint64)

In [12]:
density = cic_3d(points, ndims)

In [13]:
veldensity = cic_3d(points, ndims, vel) / len(points)

In [19]:
len(points)

16777216

In [81]:
cic.plot_density(veldensity[:,:,:,0])

In [41]:
np.max(veldensity)

0.29228416308296901

In [42]:
import scipy.fftpack as fftpack

def shearfft(vel_density):
    fftpack.fft2(vel_density[:,:,:])

In [70]:
fftpack.fft(fftpack.ifft(veldensity[:,24,56,0])*np.arange(130)*2j*math.pi)

array([-0.04283642 -1.29909372e-02j, -0.04402482 -2.29854669e-02j,
       -0.04143077 -2.11024394e-02j, -0.04187869 -1.82031345e-02j,
       -0.04455878 -1.70555271e-02j, -0.04838235 -1.80164126e-02j,
       -0.05297244 -2.06422916e-02j, -0.06112545 -2.24556815e-02j,
       -0.07721862 -3.35903773e-02j, -0.05374225 -1.26101479e-01j,
        0.00638753 -1.11094644e-01j,  0.01934295 -2.43240908e-02j,
       -0.00146231 -1.76631053e-02j, -0.01156810 -2.32226215e-02j,
       -0.01209121 -3.07944176e-02j, -0.00964803 -2.64552958e-02j,
       -0.01123140 -1.96416684e-02j, -0.01497217 -1.79808503e-02j,
       -0.01849293 -1.72045598e-02j, -0.02173493 -1.75989692e-02j,
       -0.02477827 -1.76088830e-02j, -0.02862274 -1.66207884e-02j,
       -0.03490642 -1.45106325e-02j, -0.04465997 -1.40858880e-02j,
       -0.05980322 -1.44276208e-02j, -0.09046169 -1.46970646e-02j,
       -0.12910221 -7.17530812e-02j, -0.06312521 -2.86329573e-01j,
        0.07314016 -2.33906507e-01j,  0.11605061 -7.98932836e-

In [31]:
x = np.linspace(0,2*np.pi,256,endpoint=False)
testfft = np.cos(x)

In [14]:
x = np.arange(256)
testfft = np.array([16.]*128 + [0.]*128)

In [148]:
x = np.arange(256)
testfft = np.linspace(-1, 1, 256)

In [10]:
x = np.linspace(0,2*np.pi,257,endpoint=False)
testfft = np.sin(x) + np.cos(5*x)

In [190]:
x = np.linspace(0,2*np.pi,256,endpoint=False)
testfft = np.sin(8.5*x) + np.sin(-7.5*x)

In [194]:
x = np.arange(256)
testfft = np.random.random(256)

In [15]:
plt.plot(x, testfft)
plt.show()

In [162]:
resultfft = fftpack.fft(testfft)
difft = fftpack.ifft(2j * math.pi * np.concatenate((np.arange(0,127), np.zeros(1), np.arange(-128,0))) / (2 * math.pi) * resultfft)
difft

ValueError: operands could not be broadcast together with shapes (256,) (257,) 

In [131]:
resultrfft = fftpack.rfft(testfft)
shifty(resultrfft)
resultrfft
resultrfft *  np.concatenate((np.zeros(1), np.hstack((np.arange(1, 128),np.arange(1, 128))), np.zeros(1)))
difft = fftpack.irfft(resultrfft)
difft

array([-0.00798027, -0.03193933, -0.05606986, -0.08022837, -0.10440019,
       -0.12854485, -0.15264771, -0.17668306, -0.20063631, -0.22448677,
       -0.24821996, -0.27181754, -0.29526519, -0.31854596, -0.34164571,
       -0.3645484 , -0.38724015, -0.40970564, -0.43193123, -0.45390221,
       -0.47560524, -0.49702616, -0.51815196, -0.538969  , -0.55946464,
       -0.57962574, -0.59944008, -0.61889501, -0.63797874, -0.65667916,
       -0.67498493, -0.69288447, -0.71036692, -0.72742127, -0.74403717,
       -0.76020417, -0.77591247, -0.79115219, -0.8059141 , -0.82018894,
       -0.83396805, -0.84724279, -0.86000511, -0.87224703, -0.88396111,
       -0.89514001, -0.90577695, -0.91586528, -0.92539886, -0.93437172,
       -0.94277841, -0.95061365, -0.95787269, -0.96455095, -0.97064439,
       -0.97614914, -0.98106187, -0.98537944, -0.98909924, -0.99221888,
       -0.99473645, -0.9966503 , -0.99795928, -0.99866245, -0.9987594 ,
       -0.99824995, -0.9971344 , -0.99541334, -0.99308779, -0.99

In [33]:
difft = fft_deriv(testfft, 2 * math.pi)
difft

array([ -9.69969790e-16,  -2.45412285e-02,  -4.90676743e-02,
        -7.35645636e-02,  -9.80171403e-02,  -1.22410675e-01,
        -1.46730474e-01,  -1.70961889e-01,  -1.95090322e-01,
        -2.19101240e-01,  -2.42980180e-01,  -2.66712757e-01,
        -2.90284677e-01,  -3.13681740e-01,  -3.36889853e-01,
        -3.59895037e-01,  -3.82683432e-01,  -4.05241314e-01,
        -4.27555093e-01,  -4.49611330e-01,  -4.71396737e-01,
        -4.92898192e-01,  -5.14102744e-01,  -5.34997620e-01,
        -5.55570233e-01,  -5.75808191e-01,  -5.95699304e-01,
        -6.15231591e-01,  -6.34393284e-01,  -6.53172843e-01,
        -6.71558955e-01,  -6.89540545e-01,  -7.07106781e-01,
        -7.24247083e-01,  -7.40951125e-01,  -7.57208847e-01,
        -7.73010453e-01,  -7.88346428e-01,  -8.03207531e-01,
        -8.17584813e-01,  -8.31469612e-01,  -8.44853565e-01,
        -8.57728610e-01,  -8.70086991e-01,  -8.81921264e-01,
        -8.93224301e-01,  -9.03989293e-01,  -9.14209756e-01,
        -9.23879533e-01,

In [17]:
plt.plot(x, testfft)
plt.plot(x, difft.real)
plt.plot(x, difft.imag)
plt.show()

In [55]:
timeit result = 2j * math.pi * np.concatenate((np.arange(0,127), np.zeros(1), np.arange(-128,0))) * fftpack.fft(testfft); result[-1]=0; fftpack.ifft(result)

10000 loops, best of 3: 26.5 µs per loop


In [53]:
timeit fft_deriv(testfft)

10000 loops, best of 3: 20.4 µs per loop


In [106]:
timeit y * 1j

100000 loops, best of 3: 4.21 µs per loop


In [105]:
timeit shifty(y)

1000000 loops, best of 3: 1.03 µs per loop


In [104]:
y = np.arange(1024, dtype=float)
shifty(y)
print(y)

[  0.00000000e+00  -2.00000000e+00   1.00000000e+00 ...,  -1.02200000e+03
   1.02100000e+03   1.02300000e+03]


In [56]:
(np.arange(10) + 1) // 2

array([0, 1, 1, 2, 2, 3, 3, 4, 4, 5])

In [170]:
plt.plot(np.arange(512), difft/512)
plt.show()

In [15]:
%%cython -a
from __future__ import division, print_function
import math
import numpy as np
import scipy.fftpack as fftpack
cimport numpy as cnp
cimport cython
from libc.math cimport M_PI

# Define types to be used as array dtypes
FLOAT = np.float
INT = np.int_
UINT = np.uint
ctypedef cnp.float_t FLOAT_t
ctypedef cnp.int_t INT_t
ctypedef cnp.uint_t UINT_t

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef inline void shifty(FLOAT_t[:] y):
    """Shift elements of y around as if all the complex frequencies have been multiplied by 1j"""
    cdef size_t i, j
    cdef size_t n = y.shape[0]
    # loop should be 1, 3, 5, ..., N-1 if odd, 1, 3, 5, ..., N-2 if even.
    for i in range(1, n-1, 2):
        j = i + 1
        y[i], y[j] = -y[j], y[i]

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef inline void freqy(FLOAT_t[:] y):
    """Multiply elements of y with their corresponding factors"""
    cdef size_t i
    cdef size_t n = y.shape[0]
    y[0] == 0
    for i in range(n):
        # In the original complex version, y[k] should be multiplied by 2j*pi*k/P
        # now, i + i % 2 happens to equal 2k
        # so we then multiply by pi and divide by period
        y[i] *= (i + (i % 2)) * M_PI
    # The Nyquist term, if it exists, must be murdered
    if n % 2 == 0:
        y[n-1] = 0

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef cnp.ndarray[FLOAT_t, ndim=1] fft_deriv(cnp.ndarray[FLOAT_t, ndim=1] y):
    cdef cnp.ndarray[FLOAT_t, ndim=1] coeffs = fftpack.rfft(y)
    cdef size_t n = coeffs.shape[0]
    cdef size_t i
    shifty(coeffs)
    freqy(coeffs)
    return fftpack.irfft(coeffs)

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef cnp.ndarray[FLOAT_t, ndim=5] calc_jacobian(FLOAT_t[:,:,:,:] vel_density):
    cdef size_t x_len = vel_density.shape[0]
    cdef size_t y_len = vel_density.shape[1]
    cdef size_t z_len = vel_density.shape[2]
    cdef size_t ndim = vel_density.shape[3]
    if ndim != 3:
        raise ValueError("vel_density.shape[3] must be 3")
    cdef cnp.ndarray[FLOAT_t, ndim=5] jacobian = np.zeros((x_len, y_len, z_len, ndim, ndim), dtype=FLOAT)
#    cdef FLOAT_t[:,:,:,:,:] jacobian_view = shear_tensor
    cdef size_t i, j, x, y, z
    cdef FLOAT_t[:] result
    cdef cnp.ndarray[FLOAT_t, ndim=1] vel_slice
    cdef FLOAT_t[:] vel_slice_view
    # Iterate over v_i
    for i in range(ndim):
        # iterate over x_j
        j = 0 # fft along x axis
        vel_slice = np.zeros(x_len)
        for y in range(y_len):
            for z in range(z_len):
                for x in range(x_len):
                    vel_slice[x] = vel_density[x,y,z,i]
                result = fft_deriv(vel_slice)
                # Pack result into shear tensor
                for x in range(x_len):
                    jacobian[x,y,z,i,j] = result[x]
        j = 1 # fft along y axis
        vel_slice = np.zeros(y_len)
        for x in range(y_len):
            for z in range(z_len):
                for y in range(y_len):
                    vel_slice[y] = vel_density[x,y,z,i]
                result = fft_deriv(vel_slice)
                # Pack result into shear tensor
                for y in range(y_len):
                    jacobian[x,y,z,i,j] = result[y]
        j = 2 # fft along z axis
        vel_slice = np.zeros(z_len)
        for x in range(y_len):
            for y in range(z_len):
                for z in range(z_len):
                    vel_slice[z] = vel_density[x,y,z,i]
                result = fft_deriv(vel_slice)
                # Pack result into shear tensor
                for z in range(z_len):
                    jacobian[x,y,z,i,j] = result[z]
    return jacobian

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef void calc_shear_tensor_in_place(cnp.ndarray[FLOAT_t, ndim=5] jacobian):
    cdef size_t x_len = jacobian.shape[0]
    cdef size_t y_len = jacobian.shape[1]
    cdef size_t z_len = jacobian.shape[2]
    cdef size_t i_len = jacobian.shape[3]
    cdef size_t j_len = jacobian.shape[4]
    for x in range(x_len):
        for y in range(y_len):
            for z in range(z_len):
                for i in range(i_len):
                    jacobian[x,y,z,i,i] *= 2
                    for j in range(i):
                        jacobian[x,y,z,i,j] += jacobian[x,y,z,j,i]
                        jacobian[x,y,z,j,i] = jacobian[x,y,z,i,j]

@cython.wraparound(False)
@cython.boundscheck(False)
def eig_shear_tensor(cnp.ndarray[FLOAT_t, ndim=5] jacobian):
    cdef size_t x_len = jacobian.shape[0]
    cdef size_t y_len = jacobian.shape[1]
    cdef size_t z_len = jacobian.shape[2]
    cdef size_t ndim = jacobian.shape[3]
    cdef cnp.ndarray[FLOAT_t, ndim=4] eigvals = np.zeros((x_len, y_len, z_len, ndim), dtype=FLOAT)
    for x in range(x_len):
        for y in range(y_len):
            for z in range(z_len):
                pass

In [134]:
timeit jacobian = calc_jacobian(veldensity); calc_shear_tensor_in_place(jacobian)

1 loops, best of 3: 3.46 s per loop


In [17]:
shear = calc_jacobian(veldensity)
calc_shear_tensor_in_place(shear)

In [138]:
shear[23,34,65] + shear[23,34,65].T

array([[ 0.00429444,  0.02348647, -0.00591156],
       [ 0.02348647,  0.01582374, -0.00812439],
       [-0.00591156, -0.00812439,  0.00162988]])

In [18]:
shear[23,34,65]

array([[ 0.00429444,  0.02348647, -0.00591156],
       [ 0.02348647,  0.01582374, -0.00812439],
       [-0.00591156, -0.00812439,  0.00162988]])

In [19]:
shear_eigvals = np.array([np.linalg.eigvalsh(shear[x,y,z]) for x in xrange(shear.shape[0]) for y in xrange(shear.shape[1]) for z in xrange(shear.shape[2])])

In [67]:
pos_eigval_count = np.array(((shear_eigvals/200) > 0.01).sum(axis=1)).reshape(shear.shape[:3])
pos_eigval_count

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ..., 
       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, 

In [58]:
pos_eigval_count = pos_eigval_count.reshape(shear.shape[:3])

In [37]:
np.max(shear_eigvals/200)

0.5473334645599518

In [68]:
cic.plot_density(pos_eigval_count)
cic.plot_density(density.reshape(density.shape[:3]))